In [52]:
import nba_api
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
from datetime import date
import numpy as np

In [2]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players

In [3]:
seasons = list()
for s in range(1995, 2024):
    seasons.append(str(s) + "-" + str(s+1)[-2:])

In [4]:
def print_progress_bar(iteration, total, length=50):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = '#' * filled_length + '-' * (length - filled_length)
    sys.stdout.write(f'\rProgress: [{bar}] {percent}% Complete')
    sys.stdout.flush()

In [5]:
player_df = pd.DataFrame(players.get_players())

In [6]:
active_player_df = player_df[player_df.is_active]

In [7]:
game_logs = pd.DataFrame()
active_player_ids = active_player_df["id"]

In [8]:
errors = list()
num_errors = 0
for i, player in enumerate(active_player_ids):
    
    player_seasons = list()
    
    try:
        player_career = playercareerstats.PlayerCareerStats(player_id=player).get_data_frames()[0]
        player_seasons = player_career['SEASON_ID'].unique()
    except:
        errors.append(player)
    
    for season in player_seasons:
        
        try:
            mini_log = playergamelog.PlayerGameLog(player_id=player, season=season).get_data_frames()[0]
        except:
            errors.append((player, season))
            
        game_logs = pd.concat([game_logs, mini_log])
    
    print_progress_bar(i + 1, len(active_player_ids))
    
    if len(errors) != num_errors:
        sys.stdout.write('\r' + str(errors[(num_errors-len(errors)):]) + "".join(["\t"]*80))
        sys.stdout.flush()
        num_errors = len(errors)
        print()

[(1629028, '2021-22'), (1629028, '2022-23')]																																																																																
[(1628962, '2020-21')]																																																																																
[(1628963, '2018-19'), (1628963, '2020-21')]																																																																																
[(1627736, '2016-17'), (1627736, '2017-18'), (1627736, '2018-19'), (1627736, '2020-21'), (1627736, '2022-23')]																																																																																
[1630699]																																																																																
[202722]																																																																																
[(201976, '2012-13'), (201976, '2013-14'), (201976, '2014-15'), (201976, '2015-16'), (201976, '2017-18'), (201976, '2018-19'), (201976, '2019-20')]																																																																																
Progress: [######################################

In [9]:
game_logs

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,1630173,0022001069,"MAY 16, 2021",MIA @ DET,W,42,10,16,0.625,...,7,10,0,2,1,2,3,23,6,1
1,22020,1630173,0022001062,"MAY 15, 2021",MIA @ MIL,L,2,1,2,0.500,...,0,0,1,0,0,0,0,2,3,1
2,22020,1630173,0022001050,"MAY 13, 2021",MIA vs. PHI,W,2,0,0,0.000,...,0,0,0,0,0,0,0,0,-5,1
3,22020,1630173,0022000986,"MAY 04, 2021",MIA vs. DAL,L,6,1,2,0.500,...,1,1,1,0,1,0,0,2,12,1
4,22020,1630173,0022000969,"MAY 02, 2021",MIA @ CHA,W,1,0,1,0.000,...,0,0,0,0,0,0,0,0,-3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,22023,1627826,0022300127,"NOV 01, 2023",LAC @ LAL,L,29,3,8,0.375,...,7,9,0,0,0,1,5,7,-6,1
64,22023,1627826,0022300114,"OCT 31, 2023",LAC vs. ORL,W,23,4,8,0.500,...,7,9,1,0,1,1,1,9,19,1
65,22023,1627826,0022300099,"OCT 29, 2023",LAC vs. SAS,W,22,6,7,0.857,...,6,8,0,0,1,1,4,16,23,1
66,22023,1627826,0022300085,"OCT 27, 2023",LAC @ UTA,L,19,3,6,0.500,...,2,2,0,0,2,1,2,6,1,1


In [10]:
errors

[(1629028, '2021-22'),
 (1629028, '2022-23'),
 (1628962, '2020-21'),
 (1628963, '2018-19'),
 (1628963, '2020-21'),
 (1627736, '2016-17'),
 (1627736, '2017-18'),
 (1627736, '2018-19'),
 (1627736, '2020-21'),
 (1627736, '2022-23'),
 1630699,
 202722,
 (201976, '2012-13'),
 (201976, '2013-14'),
 (201976, '2014-15'),
 (201976, '2015-16'),
 (201976, '2017-18'),
 (201976, '2018-19'),
 (201976, '2019-20')]

In [11]:
for error in errors:
    if type(error) == tuple:
        mini_log = playergamelog.PlayerGameLog(player_id=error[0], season=error[1]).get_data_frames()[0]
        game_logs = pd.concat([game_logs, mini_log])
    else:
        player_career = playercareerstats.PlayerCareerStats(player_id=error).get_data_frames()[0]
        player_seasons = player_career['SEASON_ID'].unique()
        for season in player_seasons:
            mini_log = playergamelog.PlayerGameLog(player_id=error, season=season).get_data_frames()[0]
            game_logs = pd.concat([game_logs, mini_log])

In [12]:
game_logs

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,1630173,0022001069,"MAY 16, 2021",MIA @ DET,W,42,10,16,0.625,...,7,10,0,2,1,2,3,23,6,1
1,22020,1630173,0022001062,"MAY 15, 2021",MIA @ MIL,L,2,1,2,0.500,...,0,0,1,0,0,0,0,2,3,1
2,22020,1630173,0022001050,"MAY 13, 2021",MIA vs. PHI,W,2,0,0,0.000,...,0,0,0,0,0,0,0,0,-5,1
3,22020,1630173,0022000986,"MAY 04, 2021",MIA vs. DAL,L,6,1,2,0.500,...,1,1,1,0,1,0,0,2,12,1
4,22020,1630173,0022000969,"MAY 02, 2021",MIA @ CHA,W,1,0,1,0.000,...,0,0,0,0,0,0,0,0,-3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,22019,201976,0021900063,"OCT 30, 2019",LAC @ UTA,L,25,4,13,0.308,...,2,3,0,1,0,2,0,8,-13,2
47,22019,201976,0021900051,"OCT 28, 2019",LAC vs. CHA,W,31,2,9,0.222,...,4,5,4,2,0,3,3,5,24,1
48,22019,201976,0021900035,"OCT 26, 2019",LAC @ PHX,L,30,4,8,0.500,...,5,7,1,2,0,2,6,11,-6,1
49,22019,201976,0021900016,"OCT 24, 2019",LAC @ GSW,W,28,2,4,0.500,...,4,4,3,5,0,2,3,5,20,1


In [13]:
active_player_df = active_player_df.rename(columns={"id": "Player_ID"})
active_player_df = active_player_df.drop(columns=["first_name", "last_name", "is_active"])

In [14]:
game_logs = game_logs.merge(active_player_df, how='left', on="Player_ID")

In [15]:
game_logs = game_logs.rename(columns={"full_name": "PLAYER_NAME"})

In [16]:
game_logs["SEASON"] = game_logs["SEASON_ID"].str[1:].astype(int)

In [17]:
game_logs.sort_values("SEASON", ascending=False)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PLAYER_NAME,SEASON
125163,22023,1630202,0022301034,"MAR 25, 2024",BOS @ ATL,L,28,7,12,0.583,...,6,0,0,0,3,15,-5,1,Payton Pritchard,2023
36353,22023,201942,0022301091,"APR 01, 2024",CHI vs. ATL,L,39,10,23,0.435,...,5,2,0,3,4,31,-6,1,DeMar DeRozan,2023
36355,22023,201942,0022301066,"MAR 29, 2024",CHI @ BKN,L,37,11,20,0.550,...,2,1,1,3,2,31,-2,1,DeMar DeRozan,2023
36356,22023,201942,0022301054,"MAR 27, 2024",CHI vs. IND,W,38,9,18,0.500,...,2,1,0,0,3,27,19,1,DeMar DeRozan,2023
36357,22023,201942,0022301037,"MAR 25, 2024",CHI vs. WAS,L,38,9,22,0.409,...,6,5,1,3,1,27,9,1,DeMar DeRozan,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73180,22003,2544,0020300830,"FEB 25, 2004",CLE @ HOU,L,39,5,18,0.278,...,5,2,1,2,1,11,-5,0,LeBron James,2003
73179,22003,2544,0020300838,"FEB 27, 2004",CLE @ ORL,W,49,11,19,0.579,...,3,3,1,5,1,30,7,0,LeBron James,2003
73178,22003,2544,0020300865,"MAR 01, 2004",CLE @ CHI,L,41,7,17,0.412,...,3,4,1,3,2,18,-4,0,LeBron James,2003
73177,22003,2544,0020300876,"MAR 03, 2004",CLE vs. ATL,W,28,9,17,0.529,...,3,1,1,1,0,24,22,0,LeBron James,2003


In [18]:
game_logs['GAME_DATE']= pd.to_datetime(game_logs['GAME_DATE'])

/var/folders/5d/x77440zn3c332t6t3c7wdsnm0000gn/T/ipykernel_68472/903251678.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  game_logs['GAME_DATE']= pd.to_datetime(game_logs['GAME_DATE'])


In [19]:
game_logs = game_logs.sort_values("GAME_DATE", ascending=False)

In [20]:
game_logs.to_csv("game_logs.csv")

In [21]:
game_logs

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PLAYER_NAME,SEASON
56120,22023,1629060,0022301195,2024-04-14,LAL @ NOP,W,34,5,10,0.500,...,4,2,0,1,1,11,10,1,Rui Hachimura,2023
153178,22023,1631104,0022301197,2024-04-14,SAS vs. DET,W,30,7,9,0.778,...,5,3,0,1,2,17,24,1,Blake Wesley,2023
142343,22023,1630560,0022301192,2024-04-14,BKN @ PHI,L,27,5,12,0.417,...,2,0,0,1,0,18,-4,1,Cam Thomas,2023
14868,22023,203992,0022301188,2024-04-14,ATL @ IND,L,27,6,15,0.400,...,1,2,0,0,2,18,-31,1,Bogdan Bogdanovic,2023
29119,22023,1628380,0022301197,2024-04-14,SAS vs. DET,W,16,6,11,0.545,...,2,1,0,0,3,17,18,1,Zach Collins,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73230,22003,2544,0020300068,2003-11-07,CLE @ IND,L,44,8,18,0.444,...,3,0,0,7,2,23,-7,0,LeBron James,2003
73231,22003,2544,0020300057,2003-11-05,CLE vs. DEN,L,41,3,11,0.273,...,7,2,3,2,1,7,-3,0,LeBron James,2003
73232,22003,2544,0020300038,2003-11-01,CLE @ POR,L,39,3,12,0.250,...,6,2,0,2,3,8,-21,0,LeBron James,2003
73233,22003,2544,0020300018,2003-10-30,CLE @ PHX,L,41,8,17,0.471,...,8,1,0,7,1,21,-3,0,LeBron James,2003
